# **Day - 1A | From Prompt to Action (AI Agents Intensive Course with Google)**
---
This notebook is the first step into building AI agents. An agent can do more than just respond to a prompt — it can take actions to find information or get things done.

In this notebook, we'll:
 - Install `Agent Development Kit (ADK)`
 - Configure your API key to use the Gemini model
 - Build the first simple agent
 - Run the agent and watch it use a tool (like Google Search) to answer a question


### **Setup**
#### Install Dependencies

In [2]:
pip install --quiet google-adk

Note: you may need to restart the kernel to use updated packages.


#### Configure your Gemini API Key¶
This notebook uses the Gemini API, which requires authentication.

1. Get your API key - create an API key in [Google AI Studio](https://aistudio.google.com/app/api-keys).

2. Add the key to Kaggle Secrets

Next, we need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
Create a new secret with the label `GOOGLE_API_KEY`.
Paste your API key into the "Value" field and click "Save".
Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.
3. Authenticate in the notebook

In [8]:
R = "\033[0m"

B = "\033[1m"
I = "\033[3m"
U = "\033[4m"

# Foreground Colors ..
Red = "\033[31m"
Green = "\033[32m"
Yellow = "\033[33m"
Blue = "\033[34m"

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")

In [7]:
# Run this cell to complete authentication ..
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print(f"{Green}{B}Gemini API key setup complete.{R}")
except Exception as e:
    print(f"{Red}{B}Authentication Effor: Please ensure you have added 'GOOGLE_API_KEY' to your Kaggle secrets.{R}\n\nDetails : {e}")

Gemini API key setup complete.


#### Import ADK Components

Importing the specific components needed from ADk and the Gen AI Library.

In [9]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print(f"{Green}{B}ADK components imported successfully.{R}")

ADK components imported successfully.


#### Helper functions
Probably not needed if this notebook is being used outside Kaggle Environment

In [44]:
# Define helper functions that will be reused throughout the notebook ..

from IPython.core.display import display, Markdown, HTML
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment ..
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print(f"{Green}Helper functions defined.{R}")

Helper functions defined.


### **First AI Agent with ADK**
---
#### What is an AI Agent
As we have used LLM like Gemini before, where there is a text response to a prompt.

`Prompt -> LLM -> Text Output`

AI Agents can do more! They can think, take actions and obserce the results of the actions to give a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent which takes the action of searching Google.

#### **Define the Agent**
We'll configure an `Agent` by setting its key and properties, which tell it what to do and how to operate.

The main properties we'll set :
| Property         | Description                                                                                      |
|------------------|--------------------------------------------------------------------------------------------------|
| **name** and **description** | A simple name and description to identify our agent.                                         |
| **model**            | The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".      |
| **instruction**      | The agent's guiding prompt. This tells the agent its goal is and how to behave.                 |
| **tools**            | A list of tools that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online. |

In [12]:
root_agent = Agent(
    name = "helpful_assistant",
    model = "gemini-2.5-flash-lite",
    description = "A simple agent that can answer general questions.",
    instruction = "You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools = [google_search]
)

print(f"{Green}Root Agent defined.{R}")

Root Agent defined.


#### Run your Agent
Now, it's time to bring your agent to life and send it a query. To do this, you need a `Runner`, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent and handles its responses.

In simple words, The `Runner` is like a manager that helps your agent work. It sends your questions to the agent, gets the answers back, and keeps the conversation going.


In [13]:
runner = InMemoryRunner(agent = root_agent)

print(f"{Green}{B}Runner Created.{R}")

Runner Created.


In [14]:
response = await runner.run_debug("What is ADK from Google? What languages id the SDK available in?")


 ### Created new session: debug_session_id

User > What is ADK from Google? What languages id the SDK available in?
helpful_assistant > The Agent Development Kit (ADK) from Google is an open-source framework designed to simplify the development and deployment of AI agents and multi-agent systems. It aims to make agent development more akin to traditional software development by providing a flexible and modular structure. The ADK allows developers to create agents that can perform complex tasks and workflows, orchestrate multiple agents, and integrate various tools and capabilities. It is optimized for Google's Gemini models and Vertex AI but is model-agnostic and compatible with other frameworks.

The ADK is available in the following programming languages:
*   **Python**
*   **Java**
*   **Go**

More languages are expected to be supported in the future.


In [49]:
response = await runner.run_debug("What is the weather in Delhi?")


 ### Continue session: debug_session_id

User > What is the weather in Delhi?
helpful_assistant > The weather in Delhi is currently clear with a temperature around 22°C, feeling like 22°C due to humidity. The humidity is at 49%, and wind speeds are around 4.7 km/h.

The forecast for today, Monday, November 10, 2025, indicates clear skies. The high temperature is expected to be around 26°C, with a low of 12°C.

Looking ahead, the weather is expected to remain mostly clear with temperatures generally ranging between 24°C and 28°C for the upcoming week.


#### Trying the ADK Web Interface

ADK includes a built-in-web interface for interactively chatting with testing and debugging the agent.

To use the ADK web UI, we need to create an agent with Python files using the `adk create` command.

Running the below command to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for the code, an `.env` file with the API key pre-configured and an `__init__.py` file.

In [51]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



In [54]:
# Getting the custom URL to access ADK web UI..
url_prefix = get_adk_proxy_url()

In [55]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [166]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http:

Note: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.

‼️ ***IMPORTANT: DO NOT SHARE THE PROXY LINK with anyone - treat it as sensitive data as it contains your authentication token in the URL.***

#### Next Steps
Continue to the next notebook ***Day - 1B | Agent Architectures*** to learn how to architect multi-agent systems.